<a href="https://colab.research.google.com/github/Freperush/Apps-web/blob/master/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d dansbecker/food-101
! pip show kaggle
! mkdir train
! unzip /content/food-101.zip -d train

In [30]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import tracemalloc

# Iniciar el rastreo del consumo de memoria
tracemalloc.start()

dataset_dir = '/content/train/food-101/food-101/images'
clases = os.listdir(dataset_dir)
num_categorias = len(clases)
batch_size = 100
train_ratio = 0.8

caracteristicas = []
etiquetas = []

# Recorrer cada clase del dataset
for i, clase in enumerate(clases):
    #print("Procesando categoría", i+1, "de", num_categorias)
    # Ruta a las imágenes de la clase actual
    clase_dir = os.path.join(dataset_dir, clase)

    # Verificar si es un directorio
    if not os.path.isdir(clase_dir):
        continue

    # Recorrer cada imagen de la clase
    imagen_nombres = os.listdir(clase_dir)
    num_imagenes = len(imagen_nombres)
    num_imagenes = min(num_imagenes, 100)  # Limitar a 100 imágenes por categoría
    # Contador de imágenes procesadas
    imagenes_procesadas = 0

    # Procesar las imágenes por lotes
    for batch_start in range(0, num_imagenes, batch_size):
        batch_end = min(batch_start + batch_size, num_imagenes)
        batch_imagenes = imagen_nombres[batch_start:batch_end]


        for imagen_nombre in batch_imagenes:
            # Ruta completa de la imagen
            imagen_ruta = os.path.join(clase_dir, imagen_nombre)

            # Leer la imagen y redimensionarla a un tamaño adecuado
            imagen = imread(imagen_ruta)
            imagen = resize(imagen, (224, 224))

            # Convertir la imagen en un vector de características
            caracteristica = imagen.flatten()
            caracteristicas.append(caracteristica)

            # Agregar la etiqueta correspondiente
            etiquetas.append(i)

            # Incrementar el contador de imágenes procesadas
            imagenes_procesadas += 1

            # Imprimir el progreso
            #print("Procesando imagen", imagenes_procesadas, "de", num_imagenes, "de la categoría", i+1, "- Imágenes restantes:", num_imagenes - imagenes_procesadas)

# Detener el rastreo del consumo de memoria
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

# Imprimir las estadísticas del consumo de memoria
print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

# Convertir las listas en arrays numpy
caracteristicas = np.array(caracteristicas)
etiquetas = np.array(etiquetas)
# print(caracteristicas)
# print(etiquetas)


KeyboardInterrupt: ignored

In [36]:
del caracteristicas

In [37]:
%whos


Variable              Type         Data/Info
--------------------------------------------
accuracy_score        function     <function accuracy_score at 0x7fe25f8edea0>
batch_end             int          100
batch_imagenes        list         n=100
batch_size            int          100
batch_start           int          0
clase                 str          shrimp_and_grits
clase_dir             str          /content/train/food-101/f<...>1/images/shrimp_and_grits
clases                list         n=102
dataset_dir           str          /content/train/food-101/food-101/images
etiquetas             list         n=6443
files                 module       <module 'google.colab.fil<...>s/google/colab/files.py'>
i                     int          65
imagen                ndarray      512x512x3: 786432 elems, type `uint8`, 786432 bytes (768.0 kb)
imagen_nombre         str          2331485.jpg
imagen_nombres        list         n=1000
imagen_ruta           str          /content/train/food-101

In [ ]:
%whos


In [39]:
import sys

# Obtener el tamaño de todas las variables definidas en el entorno actual
variables = dir()
for variable in variables:
    if not variable.startswith('_'):  # Ignorar variables internas
        size = sys.getsizeof(eval(variable))
        print(f"Tamaño de {variable}: {size} bytes")


Tamaño de In: 376 bytes
Tamaño de Out: 232 bytes
Tamaño de accuracy_score: 144 bytes
Tamaño de batch_end: 28 bytes
Tamaño de batch_imagenes: 856 bytes
Tamaño de batch_size: 28 bytes
Tamaño de batch_start: 24 bytes
Tamaño de clase: 65 bytes
Tamaño de clase_dir: 105 bytes
Tamaño de clases: 920 bytes
Tamaño de dataset_dir: 88 bytes
Tamaño de etiquetas: 53080 bytes
Tamaño de exit: 48 bytes
Tamaño de files: 72 bytes
Tamaño de get_ipython: 64 bytes
Tamaño de i: 28 bytes
Tamaño de imagen: 144 bytes
Tamaño de imagen_nombre: 60 bytes
Tamaño de imagen_nombres: 8856 bytes
Tamaño de imagen_ruta: 117 bytes
Tamaño de imagenes_procesadas: 28 bytes
Tamaño de imread: 144 bytes
Tamaño de memory_info: 128 bytes
Tamaño de mi_funcion: 144 bytes
Tamaño de np: 72 bytes
Tamaño de num_categorias: 28 bytes
Tamaño de num_imagenes: 28 bytes
Tamaño de objgraph: 72 bytes
Tamaño de os: 72 bytes
Tamaño de process: 48 bytes
Tamaño de processes: 104 bytes
Tamaño de profile: 144 bytes
Tamaño de psutil: 72 bytes
Tamaño d

In [43]:
!pip install psutil
import psutil

# Obtener información sobre el uso de la memoria
memory_info = psutil.virtual_memory()

# Imprimir la información
print(f"Uso de memoria RAM: {memory_info.used / 1024 / 1024:.2f} MB")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Uso de memoria RAM: 8399.12 MB


In [44]:
import psutil

# Obtener la lista de procesos y su uso de memoria
processes = psutil.process_iter(['pid', 'name', 'memory_info'])

# Imprimir información de cada proceso
for process in processes:
    print(f"PID: {process.info['pid']}, Nombre: {process.info['name']}, Uso de memoria: {process.info['memory_info'].rss / 1024 / 1024:.2f} MB")


PID: 1, Nombre: docker-init, Uso de memoria: 0.01 MB
PID: 7, Nombre: node, Uso de memoria: 58.58 MB
PID: 19, Nombre: tail, Uso de memoria: 0.50 MB
PID: 30, Nombre: oom_monitor.sh, Uso de memoria: 3.15 MB
PID: 32, Nombre: run.sh, Uso de memoria: 1.78 MB
PID: 34, Nombre: kernel_manager_proxy, Uso de memoria: 14.00 MB
PID: 60, Nombre: python3, Uso de memoria: 0.00 MB
PID: 61, Nombre: colab-fileshim., Uso de memoria: 42.70 MB
PID: 78, Nombre: jupyter-noteboo, Uso de memoria: 149.72 MB
PID: 79, Nombre: dap_multiplexer, Uso de memoria: 9.77 MB
PID: 287, Nombre: python3, Uso de memoria: 7784.83 MB
PID: 318, Nombre: python3, Uso de memoria: 15.15 MB
PID: 17376, Nombre: language_service, Uso de memoria: 14.37 MB
PID: 17381, Nombre: node, Uso de memoria: 183.76 MB
PID: 18031, Nombre: sleep, Uso de memoria: 0.50 MB


In [41]:
import psutil

# Obtener el PID del proceso de Python
pid = 287  # Reemplaza con tu PID específico

# Obtener información detallada sobre el proceso
process = psutil.Process(pid)

# Obtener el uso de memoria del proceso
memory_info = process.memory_info()

# Imprimir la información de uso de memoria
print("Uso de memoria del proceso:")
print(f"RSS (Resident Set Size): {memory_info.rss / 1024 / 1024:.2f} MB")
print(f"VMS (Virtual Memory Size): {memory_info.vms / 1024 / 1024:.2f} MB")


Uso de memoria del proceso:
RSS (Resident Set Size): 7784.83 MB
VMS (Virtual Memory Size): 9004.15 MB


In [46]:
pmap -x 287


SyntaxError: ignored